# CS155 Miniproject 1

zchen@caltech.edu

Sentiment Analysis via Neural Networks

In [26]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import sklearn.model_selection

import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, LeakyReLU

## Loading Dataset

In [11]:
# Load final validation data set
FV_data = np.loadtxt('../data/test_data.txt',delimiter=' ',skiprows=1)
# Load input training data set
train_data = np.loadtxt('../data/training_data.txt',delimiter=' ',skiprows=1)

# Splot y_train and x_train from training set
y_tall = train_data[:,0]
x_tall = train_data[:,1:]

In [12]:
# One hot encode categories
y_tall = keras.utils.np_utils.to_categorical(y_tall)

# Split the training data k-fold number of ways for k-fold validation of the learning algorithm
kf = sklearn.model_selection.KFold(n_splits=5)
inds = [ind for ind in kf.split(x_tall, y_tall)]

# Returns indices for validation and training splits
train,val = inds[0]
Xtrain = x_tall[train]
Ytrain = y_tall[train]
Xval = x_tall[val]
Yval = y_tall[val]

## Trying Neural Networks on bag of words dataset

In [43]:
# Function to generate DNN of given depth and width
def getModel(layers,Pdrop):
    model = Sequential()
    model.add(Dense(layers[0],input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(Activation(LeakyReLU(alpha=0.3)))
    for i in layers[1:]:
        model.add(Dense(i))
        model.add(BatchNormalization())
        model.add(Activation(LeakyReLU(alpha=0.3)))
        model.add(Dropout(Pdrop))
    
    # predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model

# Model for problem 2C, create DNN with less than 100 hidden layers layers
model = getModel([500,250,125,75,25],0.4)
# Printing a summary of the layers and weights in your model
model.summary()

/usr/local/lib/python3.6/site-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_65 (Dense)             (None, 500)               500500    
_________________________________________________________________
batch_normalization_53 (Batc (None, 500)               2000      
_________________________________________________________________
activation_63 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 250)               125250    
_________________________________________________________________
batch_normalization_54 (Batc (None, 250)               1000      
_________________________________________________________________
activation_64 (Activation)   (None, 250)               0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 250)               0         
__________

In [44]:
# Compile the DNN model
model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
fit = model.fit(Xtrain, Ytrain, batch_size=2**8, epochs=4,verbose=1,validation_data=(Xval, Yval))

Train on 16000 samples, validate on 4000 samples
Epoch 1/4
16000/16000 [==============================] - 6s 361us/step - loss: 0.5276 - acc: 0.7460 - val_loss: 0.3738 - val_acc: 0.8550
Epoch 2/4
16000/16000 [==============================] - 3s 177us/step - loss: 0.3640 - acc: 0.8459 - val_loss: 0.4156 - val_acc: 0.8488
Epoch 3/4
16000/16000 [==============================] - 3s 172us/step - loss: 0.3204 - acc: 0.8684 - val_loss: 0.4005 - val_acc: 0.8515
Epoch 4/4
16000/16000 [==============================] - 3s 180us/step - loss: 0.2923 - acc: 0.8773 - val_loss: 0.4160 - val_acc: 0.8452


Plot the training loss and validation accuracy vs the number of weak decision trees

In [45]:
%%time
print('Training error',model.evaluate(x=Xtrain, y=Ytrain))
print('Test error',model.evaluate(x=Xval, y=Yval))

16000/16000 [==============================] - 2s 139us/step
Training error [0.23184707679599523, 0.9040625]
4000/4000 [==============================] - 1s 146us/step
Test error [0.4160181985497475, 0.84525]
CPU times: user 6.54 s, sys: 362 ms, total: 6.9 s
Wall time: 2.81 s


Validation accuracy plateaus at 82%